In [338]:
import os
import pandas as pd
import numpy as np

In [339]:
op_headers = ["anzahl", "code", "ebm", "kuerzel", "betriebsstaette", "text", "seite", "datum", "patientid", "vorname", "nachname", "geburtsdatum", "jahre", "scheinid", "geschlecht"]

In [340]:
m17_data = pd.read_csv("../data/M17_Data.csv")
m17_diag = pd.read_csv("../data/M17_Diag.csv")
m17_op = pd.read_csv("../data/M17_Operations.csv")
m17_op.columns = op_headers

m22_data = pd.read_csv("../data/M22_Data.csv")
m22_diag = pd.read_csv("../data/M22_Diag.csv")
m22_op = pd.read_csv("../data/M22_Operations.csv")
m22_op.columns = op_headers

m23_data = pd.read_csv("../data/M23_Data.csv")
m23_op = pd.read_csv("../data/M23_Operations.csv")
m23_diag = pd.read_csv("../data/M23_Diag.csv")
m23_op.columns = op_headers

s83_data = pd.read_csv("../data/S83_Data.csv")
s83_op = pd.read_csv("../data/S83_Operations.csv")
s83_diag = pd.read_csv("../data/S83_Diag.csv")
s83_op.columns = op_headers

Dataframe preparatation:
1. Concatenate all sub-dataframes of the same type
2. Reset index
3. Drop the unnecessary columns to anonymize and reduce dimensions
4. (Optional) Add any additionaly columns to make patient records, operations and diagnoses similar in format
5. Sort by date
6. (Optional) Filter a column
7. Remove any duplicates and optionally ignore a column while doing it

In [341]:
def prep_fs_data(to_concat, to_drop, to_add="", to_filter=[], to_ignore_in_duplicates=[]):
    df = pd.concat(to_concat, ignore_index=True)
    df = df.reset_index(drop=True)
    df = df.drop(to_drop, axis ="columns", inplace=False)
    if len(to_add) > 0: df[to_add] = ""
    if "datum" in df.columns:
        df["datum"] = pd.to_datetime(df["datum"], dayfirst=True, errors="coerce")
        df = df.sort_values(by="datum")
    if len(to_filter) > 1:
        df = df[df[to_filter[0]].isin(to_filter[1])] 
    df = df.drop_duplicates(subset=df.columns.difference(to_ignore_in_duplicates))
    df = df.reset_index(drop=True)
    return df

# Patient Records

In [342]:
anam_exam = prep_df(to_concat=[m17_data, m22_data, m23_data, s83_data],
                       to_drop=["typ", "betriebsstaette", "nachname", "vorname", "letzter_nutzer", "karteieintragid", "medientyp"],
                       to_add="code",
                       to_filter=["karteityp", ["ANA", "BEF"]],
                       to_ignore_in_duplicates=["karteityp"])


In [343]:
len(anam_exam)

337555

In [344]:
print(len(anam_exam))
anam_exam = anam_exam[~anam_exam['text'].str.lower().str.contains('op-datum', na=False)]
anam_exam = anam_exam[~anam_exam['text'].str.lower().str.contains('z.n.', na=False)]
anam_exam = anam_exam[~anam_exam['text'].str.lower().str.contains('postop', na=False)]
anam_exam = anam_exam[~anam_exam['text'].str.lower().str.contains('post op', na=False)]
print(len(anam_exam))

337555
204204


In [345]:
anam_exam.head()

,datum,text,karteityp,patientid,geschlecht,code
3,1900-01-01,29.05 :FB: Anfrage der Versorgungsamtes zum ...,BEF,4972,male,
5,1960-12-22,Liebe Patientin; lieber Patient; vielen Dank f...,ANA,94398,female,
6,1996-01-03,Klinischer Befund des Oberschenkels li.deutl.V...,BEF,72,male,
8,1996-01-03,Re. Achillessehne: Deutliche Auftreibung der A...,BEF,67,male,
9,1996-01-03,Seit mehreren Monaten Beschwerden re. Achilles...,ANA,67,male,


# Operations

In [346]:
ops = prep_df(to_concat=[m17_op, m22_op, m23_op, s83_op],
                       to_drop=["anzahl", "ebm", "kuerzel", "seite", "vorname", "nachname", "scheinid", "betriebsstaette"],
                       to_add="karteityp")
ops["karteityp"] = "OP"

In [347]:
ops.head()

,code,text,datum,patientid,geburtsdatum,jahre,geschlecht,karteityp
0,5-812.6,Arthroskopische Operation am Gelenkknorpel und...,2005-04-04,55162,11.09.1979,44,female,OP
1,5-812.7,Arthroskopische Operation am Gelenkknorpel und...,2005-04-04,55153,01.10.1967,56,male,OP
2,5-812.6,Arthroskopische Operation am Gelenkknorpel und...,2005-04-04,53783,01.04.1957,67,female,OP
3,5-812.6,Arthroskopische Operation am Gelenkknorpel und...,2005-04-04,55160,14.06.1946,78,female,OP
4,5-812.6,Arthroskopische Operation am Gelenkknorpel und...,2005-04-04,55157,10.08.1988,35,female,OP


# Diagnoses

In [348]:
diag = prep_df(to_concat=[m17_diag, m22_diag, m23_diag, s83_diag],
                       to_drop=["anzahl", "kuerzel", "ddi", "vorname", "nachname", "scheinid"],
                       to_add="karteityp")
diag["karteityp"] = "DIA"
diag.rename(columns={"eigene_bezeichnung":"text"}, inplace=True)


In [349]:
diag.head()

,code,icdtext,text,datum,patientid,geburtsdatum,jahre,geschlecht,karteityp
0,M22.4 G,Chondromalacia patellae,rechts,1996-01-04,113,21.09.1981,42,female,DIA
1,M23.0- G,Meniskusganglion,Binnenschädigung des Kniegelenkes (internal de...,1996-01-04,150,29.10.1966,57,male,DIA
2,M23.0- G,Meniskusganglion,Binnenschädigung des Kniegelenkes (internal de...,1996-01-04,93,04.05.1977,47,male,DIA
3,M17.9 G,Gonarthrose; nicht näher bezeichnet,Gonarthrose re.; med. Meniskusläsion re.; lat....,1996-01-04,105,08.06.1930,94,female,DIA
4,S83.2 G,Akuter Meniskusriss,Meniskusriß; akut,1996-01-04,93,04.05.1977,47,male,DIA


In [350]:
icd_text = diag[['code', 'icdtext']].copy()
icd_text['code'] = icd_text['code'].str.strip()
icd_dict = icd_text.drop_duplicates().to_dict('records')
icd_dict = {item['code']: item['icdtext'] for item in icd_dict}

In [351]:
icd_dict #icd_code: icdtext

{'M22.4 G': 'Chondromalacia patellae',
 'M23.0- G': 'Meniskusganglion',
 'M17.9 G': 'Gonarthrose; nicht näher bezeichnet',
 'S83.2 G': 'Akuter Meniskusriss',
 'M23.3-': 'Sonstige Meniskusschädigungen',
 'M17.9 A': 'Gonarthrose; nicht näher bezeichnet',
 'M23.39 G': 'Sonstige Meniskusschädigungen: nicht näher bezeichneter Meniskus',
 'S83.53 V': 'Verstauchung und Zerrung des Kniegelenkes mit Riss des vorderen Kreuzbandes',
 'M23.23 G': 'Meniskusschädigung durch alten Riss oder alte Verletzung: sonstiger und nicht näher bezeichneter Teil des Innenmeniskus',
 'M22.2 G': 'Krankheiten im Patellofemoralbereich',
 'M23.24 G': 'Meniskusschädigung durch alten Riss oder alte Verletzung: Vorderhorn des Außenmeniskus',
 'M23.59 G': 'Chronische Instabilität des Kniegelenkes: nicht näher bezeichnetes Band ',
 'M17.1 G': 'Sonstige primäre Gonarthrose',
 'M23.33 G': 'Sonstige Meniskusschädigungen: sonstiger und nicht näher bezeichneter Teil des Innenmeniskus',
 'S83.5- Z': 'Verstauchung und Zerrung de

In [352]:
birthdays = diag[['patientid', 'geburtsdatum']].drop_duplicates().to_dict('records')
birthdays = {item['patientid']: item['geburtsdatum'] for item in birthdays}

In [353]:
birthdays # patientid: birthday

{113: '21.09.1981',
 150: '29.10.1966',
 93: '04.05.1977',
 105: '08.06.1930',
 104: '03.04.1920',
 110: '22.05.1954',
 118: '10.04.1937',
 79: '10.02.1977',
 70: '09.08.1935',
 69: '06.08.1967',
 177: '16.04.1963',
 278: '08.08.1967',
 282: '16.08.1931',
 350: '07.01.1945',
 492: '01.11.1936',
 521: '27.06.1969',
 501: '06.09.1966',
 568: '06.04.1939',
 609: '03.06.1956',
 602: '24.04.1940',
 731: '09.06.1971',
 799: '18.11.1970',
 768: '23.04.1950',
 939: '17.01.1945',
 903: '06.03.1950',
 695: '28.02.1963',
 924: '14.06.1951',
 919: '29.07.1939',
 920: '16.03.1964',
 926: '23.09.1933',
 716: '18.10.1943',
 1061: '28.12.1973',
 821: '04.03.1968',
 1163: '02.04.1961',
 909: '17.12.1957',
 900: '21.04.1925',
 842: '05.03.1969',
 1261: '28.12.1966',
 1230: '25.05.1957',
 1278: '09.10.1954',
 1275: '24.10.1948',
 1308: '25.03.1927',
 1297: '02.03.1959',
 1313: '21.05.1970',
 1130: '01.05.1976',
 1077: '25.10.1933',
 1315: '13.06.1939',
 429: '15.04.1950',
 1394: '31.03.1975',
 194: '11.0

# Merge all together

### Diagnosis final pre-processing
If the icdtext is longer than the eigenbezeichnung (text), substitute text value with the icdtext value

In [354]:
diag.head()

,code,icdtext,text,datum,patientid,geburtsdatum,jahre,geschlecht,karteityp
0,M22.4 G,Chondromalacia patellae,rechts,1996-01-04,113,21.09.1981,42,female,DIA
1,M23.0- G,Meniskusganglion,Binnenschädigung des Kniegelenkes (internal de...,1996-01-04,150,29.10.1966,57,male,DIA
2,M23.0- G,Meniskusganglion,Binnenschädigung des Kniegelenkes (internal de...,1996-01-04,93,04.05.1977,47,male,DIA
3,M17.9 G,Gonarthrose; nicht näher bezeichnet,Gonarthrose re.; med. Meniskusläsion re.; lat....,1996-01-04,105,08.06.1930,94,female,DIA
4,S83.2 G,Akuter Meniskusriss,Meniskusriß; akut,1996-01-04,93,04.05.1977,47,male,DIA


In [355]:
diag_final = diag.copy()

diag_final['icdtext'] = diag_final['icdtext'].astype(str)
diag_final['text'] = diag_final['text'].astype(str)

diag_final['text'] = diag_final.apply(lambda row: row['icdtext'] if len(row['icdtext']) > len(row['text']) else row['text'], axis=1)
diag_final = diag_final.drop(columns=["icdtext"])

In [356]:
diag_final.head()

,code,text,datum,patientid,geburtsdatum,jahre,geschlecht,karteityp
0,M22.4 G,Chondromalacia patellae,1996-01-04,113,21.09.1981,42,female,DIA
1,M23.0- G,Binnenschädigung des Kniegelenkes (internal de...,1996-01-04,150,29.10.1966,57,male,DIA
2,M23.0- G,Binnenschädigung des Kniegelenkes (internal de...,1996-01-04,93,04.05.1977,47,male,DIA
3,M17.9 G,Gonarthrose re.; med. Meniskusläsion re.; lat....,1996-01-04,105,08.06.1930,94,female,DIA
4,S83.2 G,Akuter Meniskusriss,1996-01-04,93,04.05.1977,47,male,DIA


### Patient Records final pre-processing
Fill in missing years and birthdays

In [357]:
anam_exam_final = anam_exam.copy()

anam_exam_final['geburtsdatum'] = ""
anam_exam_final['geburtsdatum'] = anam_exam_final['patientid'].map(birthdays)
anam_exam_final['geburtsdatum'] = pd.to_datetime(anam_exam_final['geburtsdatum'], dayfirst=True)

# Calculate 'jahre'
anam_exam_final['jahre'] = (abs(anam_exam_final['datum'] - anam_exam_final['geburtsdatum'])).dt.days // 365

In [358]:
anam_exam_final.head()

,datum,text,karteityp,patientid,geschlecht,code,geburtsdatum,jahre
3,1900-01-01,29.05 :FB: Anfrage der Versorgungsamtes zum ...,BEF,4972,male,,1973-09-02,73.0
5,1960-12-22,Liebe Patientin; lieber Patient; vielen Dank f...,ANA,94398,female,,1970-08-15,9.0
6,1996-01-03,Klinischer Befund des Oberschenkels li.deutl.V...,BEF,72,male,,1976-09-03,19.0
8,1996-01-03,Re. Achillessehne: Deutliche Auftreibung der A...,BEF,67,male,,1954-07-01,41.0
9,1996-01-03,Seit mehreren Monaten Beschwerden re. Achilles...,ANA,67,male,,1954-07-01,41.0


### Operations final pre-processing

In [359]:
ops_final = ops.copy()

### Merge All

In [360]:
all = pd.concat([diag_final, ops_final, anam_exam_final], ignore_index=True)
all.reset_index()
all = all.drop(["geburtsdatum"], axis="columns")

In [361]:
grouped_lists = all.sort_values(["datum"]).groupby('patientid').agg(list)

def lists_to_dicts(row, pid):
    return [{'patientid': pid, **{col: row[col][i] for col in grouped_lists.columns}} for i in range(len(row[grouped_lists.columns[0]]))]

grouped_lists['entries'] = grouped_lists.apply(lambda row: lists_to_dicts(row, row.name), axis=1)

grouped_df = pd.DataFrame({
    'patientid': grouped_lists.index,
    'entries': grouped_lists['entries']
}).reset_index(drop=True)

In [362]:
print(len(grouped_df[grouped_df['patientid'].notna()]) / len(grouped_df))

1.0


In [363]:
grouped_df

,patientid,entries
0,61,"[{'patientid': 61, 'code': '', 'text': 'Klinis..."
1,67,"[{'patientid': 67, 'code': '', 'text': 'Re. Ac..."
2,68,"[{'patientid': 68, 'code': 'M23.23 V', 'text':..."
3,69,"[{'patientid': 69, 'code': 'M23.39 G', 'text':..."
4,70,"[{'patientid': 70, 'code': 'M17.9 A', 'text': ..."
...,...,...
42993,701652,"[{'patientid': 701652, 'code': 'M17.1 G', 'tex..."
42994,701654,"[{'patientid': 701654, 'code': '', 'text': 'Ga..."
42995,701695,"[{'patientid': 701695, 'code': 'M23.59 G', 'te..."
42996,701711,"[{'patientid': 701711, 'code': '', 'text': 'Am..."


In [364]:
from datetime import datetime, timedelta

In [365]:
# Function to group entries by the 2-year criterion
def group_entries(entries):
    if not entries:
        return []
    
    grouped_entries = []
    current_group = [entries[0]]

    for entry in entries[1:]:
        if entry['datum'] >= current_group[-1]['datum'] + timedelta(days=365*3):
            grouped_entries.append(current_group)
            current_group = [entry]
        else:
            current_group.append(entry)
    
    grouped_entries.append(current_group)
    return grouped_entries

# Apply the grouping function
grouped_df['grouped_entries'] = grouped_df['entries'].apply(group_entries)
grouped_df = grouped_df.drop(columns=['entries'])

In [366]:
grouped_df.iloc[0]['grouped_entries']

[[{'patientid': 61,
   'code': '',
   'text': 'Klinischer Befund des Unterschenkels re.: deutliche Schwellung. an der medialen Tibiakante; leichte Verhärtung der medialen Wadenmuskulatur.',
   'datum': Timestamp('1996-01-03 00:00:00'),
   'jahre': 30.0,
   'geschlecht': 'male',
   'karteityp': 'BEF'},
  {'patientid': 61,
   'code': '',
   'text': 'Zwischenanamnese: Beschwerden unverändert.',
   'datum': Timestamp('1996-01-05 00:00:00'),
   'jahre': 30.0,
   'geschlecht': 'male',
   'karteityp': 'ANA'},
  {'patientid': 61,
   'code': '',
   'text': 'Klinischer Befund des Unterschenkels re.: insgesamt besser;',
   'datum': Timestamp('1996-01-05 00:00:00'),
   'jahre': 30.0,
   'geschlecht': 'male',
   'karteityp': 'BEF'},
  {'patientid': 61,
   'code': '',
   'text': 'Immer noch Probleme re. Leiste. Gefühl als ob es "hakt".',
   'datum': Timestamp('1996-06-27 00:00:00'),
   'jahre': 30.0,
   'geschlecht': 'male',
   'karteityp': 'ANA'},
  {'patientid': 61,
   'code': '',
   'text': 'Bef.

In [367]:
sum(len(groups) for groups in grouped_df['grouped_entries'])

48229

In [368]:
structured_data = pd.DataFrame(columns=['patientid', 'sex', 'age', 'ANA', 'EXA', 'DIA_code', 'DIA_text', 'OP_code', 'OP_text'])
def has_X(type, dict):
    return dict['karteityp'] == type

def get_elements(entry, arg):
    if entry is not None: return entry[arg]
    else: return None
        

def structure_occurrence(group):
    ana_elem = next((entry for entry in group if has_X('ANA', entry)), None)
    bef_elem = next((entry for entry in group if has_X('BEF', entry)), None)
    op_elem = next((entry for entry in group if has_X('OP', entry)), None)
    dia_elem = next((entry for entry in group if has_X('DIA', entry)), None)
    
    if ana_elem is not None:
        patientid = get_elements(ana_elem, 'patientid')
        sex = get_elements(ana_elem, 'geschlecht')
        age = get_elements(ana_elem, 'jahre')
    else:
        patientid = get_elements(bef_elem, 'patientid')
        sex = get_elements(ana_elem, 'geschlecht')
        age = get_elements(ana_elem, 'jahre')
    
    ANA = get_elements(ana_elem, 'text')
    EXA = get_elements(bef_elem, 'text')
    DIA = get_elements(dia_elem, 'text')
    DIA_code = get_elements(dia_elem, 'code')
    OP = get_elements(op_elem, 'text')
    OP_code = get_elements(op_elem, 'code')
    
    return {'patientid': patientid, 'sex': sex, 'age': age, 'ANA': ANA, 'EXA': EXA, 'DIA_text': DIA, 'DIA_code': DIA_code, 'OP_text': OP, 'OP_code': OP_code}
    
flattened_entries = []
for groups in grouped_df['grouped_entries']:
    for group in groups:
        flattened_entries.append(structure_occurrence(group))

# Create a new DataFrame from the flattened entries
structured_data = pd.DataFrame(flattened_entries)

In [369]:
print(len(structured_data[structured_data['patientid'].notna()]) / len(structured_data))

0.8783719338986916


In [370]:
structured_data.EXA.str.count("OP-Datum").sum()

0.0

In [371]:
len(structured_data)

48229

In [372]:
structured_data.to_csv('../data/structured_data.csv', index=False)

In [373]:
import pandas as pd
structured_data = pd.read_csv('../data/structured_data.csv')


In [374]:
len(structured_data)

48229

In [375]:
fs_data = structured_data.copy()
fs_data = fs_data.dropna(subset=['ANA', 'EXA'], how='all')   
fs_data = fs_data[fs_data['DIA_code'].notna()]
len(fs_data)

40260

In [376]:
print(len(fs_data[fs_data['patientid'].notna()]) / len(fs_data))

1.0


In [377]:
df = fs_data
def fun(string):
    # Check if the value is not NaN using pd.notna()
    if pd.notna(string):
        if len(string) < 30:
            return np.nan
    return string

# Update 'EXA' column using the function
df['EXA'] = df['EXA'].apply(fun)

# Optionally, apply the same function to 'ANA' if needed
df['ANA'] = df['ANA'].apply(fun)

# Optionally, drop rows where both 'ANA' and 'EXA' are NaN
df = df.dropna(subset=['ANA', 'EXA'], how='all')
 
print(df.head())
print(len(df))

   patientid   sex   age                                                ANA  \
0       61.0  male  30.0         Zwischenanamnese: Beschwerden unverändert.   
1       67.0  male  41.0  Seit mehreren Monaten Beschwerden re. Achilles...   
5       72.0  male  19.0                     Zwischenanamnese: geht besser.   
7       84.0  male  57.0  Rechte Seite gut; links Druckschmerz medial. G...   
9       93.0   NaN   NaN                                                NaN   

                                                 EXA  \
0  Klinischer Befund des Unterschenkels re.: deut...   
1  Re. Achillessehne: Deutliche Auftreibung der A...   
5  Klinischer Befund des Oberschenkels li.deutl.V...   
7  Klinischer Befund des Kniegelenks li.: Drucksc...   
9  Klinischer Befund des Kniegelenks li.: Kontur ...   

                                            DIA_text  DIA_code OP_text OP_code  
0  Retropatellare Chondromalazie li; Insertionste...   M22.4 G     NaN     NaN  
1  V.a.laterale Chondromal

In [378]:
fs_data.to_csv('../data/filtered_structured_data2.csv', index=False)


In [380]:
print(len(fs_data))

40260


# Examination Interpretation

In [381]:
import pandas as pd
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/juliankraus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/juliankraus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Textual Adaptions

In [382]:
# List of strings that should not be split
do_not_split = [
    "li.",
    "bds.",
    "med.",
    "neg.",
    "lat.",
    'pos.',
    "re.",
    "mm, links",
    "mm, rechts",
    "mm, li",
    "mm, re",
    "o.B",
    "mm, verletzt",
    "mm, Seitendifferenz",
    "diskr.",
    "flüssig und mit"
]
def custom_split(text, exceptions):
    for exception in exceptions:
        text = re.sub(re.escape(exception), re.sub(r'[^\w\s]', '', exception), text, flags=re.IGNORECASE)
    parts = re.split(r'[.,;]|\sund\s', text)
    parts = [part.strip() for part in parts if part.strip()]
    return parts

In [383]:
# Preprocess the text
pattern_pre = r'\d{1,3}/\d{1,3}/\d{1,3}|\d{1,3}-\d{1,3}-\d{1,3}'

pattern_side = r'(\b\d+[\d.,]*\b)\s*seitendifferenz|\bseitendifferenz\s*(\d+[\d.,]*\b)'


def adaptText(match):
    # Extract the matched text
    matched_text = match.group(0)
    
    # Split the matched text by either '/' or '-'
    if '/' in matched_text:
        parts = matched_text.split('/')
    elif '-' in matched_text:
        parts = matched_text.split('-')
    
    # Convert parts to integers
    parts = list(map(int, parts))
    
    # Evaluate each part in the context of the "Neutral-Null-Methode"
    hyperextension = parts[0]
    extension_deficit = parts[1]
    flexion = parts[2]
    
    # Create descriptions based on the measurements
    if hyperextension > 0:
        hyperextension_desc = "überstreckung des knies ist möglich,"
    else:
        hyperextension_desc = "keine überstreckung des knies,"
    
    if extension_deficit > 0:
        extension_deficit_desc = "streckdefizit des knies,"
    else:
        extension_deficit_desc = "kein streckdefizit des knies,"
    
    if flexion >= 120:
        flexion_desc = "gute beugung des knies,"
    elif flexion >= 90:
        flexion_desc = "angemessene beugung des knies,"
    else:
        flexion_desc = "eingeschränkte beugung des knies,"
    
    # Combine the descriptions into a final string
    result = f",{hyperextension_desc} {extension_deficit_desc} {flexion_desc}"
    return result

def adaptText_2(match):
    # Extract the matched text
    matched_text = match.group(0)
    
    number = match.group(1) if match.group(1) else match.group(2)
    if int(number) > 3:
        return "wacklig"
    else:
        return ""

def preprocess(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub('ds', 'druckschmerz', text, flags=re.IGNORECASE)
    text = re.sub('medial', 'innere', text, flags=re.IGNORECASE)
    text = re.sub('gelenkspalt', 'seite des kniegelenks', text, flags=re.IGNORECASE)



    text = re.sub('lateral', 'äußere', text, flags=re.IGNORECASE)
    text = re.sub('lat', 'äußere', text, flags=re.IGNORECASE)
    text = re.sub('gs', 'seite des kniegelenks', text, flags=re.IGNORECASE)
    text = re.sub('li ', 'links ', text, flags=re.IGNORECASE)
    text = re.sub('re ', 'rechts ', text, flags=re.IGNORECASE)
    text = re.sub('gb ', 'gangbild ', text, flags=re.IGNORECASE)
    text = re.sub('patellafacette', 'kniescheibe', text, flags=re.IGNORECASE)
    text = re.sub('patella', 'kniescheibe', text, flags=re.IGNORECASE)
    text = re.sub('erguss', 'schwellung', text, flags=re.IGNORECASE)
    text = re.sub('lcp', 'fühle mich', text, flags=re.IGNORECASE)
    text = re.sub('lachman neg', '', text, flags=re.IGNORECASE)
    text = re.sub('lachman pos', 'fühle mich unsicher', text, flags=re.IGNORECASE)


    text = re.sub(pattern_side, adaptText_2, text, flags=re.IGNORECASE)


    text = re.sub(pattern_pre, adaptText, text, flags=re.IGNORECASE)

    text = re.sub(r'[^\w\s/-]', '', text)
    return text

In [384]:
patterns = [
    r'^\d{1,3}/\d{1,3}/\d{1,3}$',  # Pattern for 1-3 digits/numbers/1-3 digits/numbers/1-3 digits/numbers
    r'^\d{1,3}-\d{1,3}-\d{1,3}$',  # Pattern for 1-3 digits-numbers-1-3 digits-numbers-1-3 digits-numbers
    r'.*gangbild.*',               
    r'.*druck.*',                 
    r'.*ergu.*',              
    r'.*schwell.*',       
    r'.*rötung.*',          
    r'.*überstreckung.*',     
    r'.*fühle.*',   
    r'.*wacklig.*',     


]

# Combine patterns into a single regular expression
combined_pattern = r'|'.join(patterns)

In [385]:
inter_df = fs_data.copy()
inter_df = inter_df.fillna(value="")

# Filter the list
for idx, row in inter_df.iterrows():
    text = row['EXA']
    if (text is not None) and text != "":
        parts = custom_split(text, do_not_split)  # Custom split by exceptions
        parts = list(map(preprocess, parts))
        
        filtered_parts = [s for s in parts if s and re.match(combined_pattern, s, re.IGNORECASE)]
        
        inter_df.at[idx, 'EXA_formatted'] = ', '.join(filtered_parts)
            
    else:
        inter_df.at[idx, 'EXA_formatted'] = ""

In [386]:
midpoint = len(inter_df) // 2

# Split the DataFrame
test = inter_df.copy().iloc[:midpoint]
test2 = inter_df.copy().iloc[midpoint:]
test.to_csv("test.csv", index=False)
test.to_csv("test2.csv", index=False)

## Formulation

In [335]:
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# # Load the Mistral model and tokenizer from Hugging Face
# model_name = "jphme/em_german_leo_mistral"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)
# 
# import torch
# import pandas as pd
# import tensorflow as tf
# if torch.backends.mps.is_available():
#     mps_device = torch.device("mps")
#     print("Using MPS device:", mps_device)
#     model.to(mps_device)
# # Create a text generation pipeline
# text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0, return_full_text=False)
# few_shot_examples = [
#     ("links knie diskr erguss, kein anteromedialer druckschmerz med äußerechts Seite des Kniegelenks, kein druckschmerz kniescheiben, keine meniskuszeichen rechts knie kein erguss, druckschmerz proximal seitlich kniescheibe", 
#      "Das linke Knie hat keinen Erguss, und es gibt keine Schmerzen beim Drücken auf der äußerechts Seite des Kniegelenks und Kniescheibe. Das rechte Knie hat auch keinen Erguss, aber an in der Nähe der Seite der Kniescheibe, gibt es Schmerzen beim Drücken."),
#     ("rechts knie deutlicher erguss, keine überstreckung des knies, streckdefizit des knies, angemessene beugung des knies, keine rötung", 
#      "Das rechte Knie hat einen deutlichen Erguss, es ist nicht möglich das Knie zu überstrecken und auch nicht es komplett zu strecken. Die Beugung des Knies ist nur angemessen möglich. Das Knie ist nicht gerötet."),
# ]

## NOT CONCURRENT

In [336]:

# Construct the few-shot prompt
#instruction = "Du bist ein Patient, formuliere die Beschreibung deiner Symptome aus."
#few_shot_prompt = f"{instruction}\n\n" + "\n\n".join([f"USER: {inp}\nASSISTANT: {out}" for inp, out in few_shot_examples])
#for idx, row in inter_df.iterrows():
#    text = row['EXA_formatted']
#    if text != "":
#        # Combine the text and instruction
#        prompt = f"{few_shot_prompt}\n\nUSER: {text} \n ASSISTANT:"
#        # Generate the output
#        generated_text = text_generation_pipeline(prompt, max_new_tokens=200)[0]['generated_text']
#        output_text = generated_text
#        inter_df.at[idx, 'EXA_interpreted'] = output_text
#    else:
#        inter_df.at[idx, 'EXA_interpreted'] = ""
#inter_df.to_csv("../data/interpreted_data.csv")

## CONCURRENT

In [337]:
import os
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from concurrent.futures import ProcessPoolExecutor, as_completed

# Load the Mistral model and tokenizer from Hugging Face
model_name = "jphme/em_german_leo_mistral"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    print("Using MPS device:", mps_device)
    model.to(mps_device)
else:
    mps_device = torch.device("cpu")
    print("MPS device not available, using CPU:", mps_device)

# Create a text generation pipeline with CPU device
text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, return_full_text=False)

few_shot_examples = [
    ("links knie diskr schwellung, kein anteromedialer druckschmerz med äußerechts Seite des Kniegelenks, kein druckschmerz kniescheiben, keine meniskuszeichen rechts knie keine schwellung, druckschmerz proximal seitlich kniescheibe", 
     "Das linke Knie hat keine Schwellung, und es gibt keine Schmerzen beim Drücken auf der äußerechts Seite des Kniegelenks und Kniescheibe. Das rechte Knie hat auch keine Schwellung, aber an in der Nähe der Seite der Kniescheibe, gibt es Schmerzen beim Drücken."),
    ("rechts knie deutlicher schwellung, keine überstreckung des knies, streckdefizit des knies, angemessene beugung des knies, keine rötung", 
     "Das rechte Knie hat eine deutliche Schwellung, es ist nicht möglich das Knie zu überstrecken und auch nicht es komplett zu strecken. Die Beugung des Knies ist nur angemessen möglich. Das Knie ist nicht gerötet."),
]

def generate_response(idx, text, few_shot_prompt, text_generation_pipeline):
    if text != "" and text is not None:
        try:
            prompt = f"{few_shot_prompt}\n\nUSER: {text}\nASSISTANT:"
            generated_text = text_generation_pipeline(prompt, max_new_tokens=200)[0]['generated_text']
            return idx, generated_text
        except Exception as e:
            print(f"Error generating response for index {idx}: {e}")
            return idx, ""
    return idx, ""

# Construct the few-shot prompt
instruction = "Du bist ein Patient mit den folgenden Symptomen. Ersetze alle lateinischen und deutschen Fachbegriffe und formuliere es umgangssprachlich."
few_shot_prompt = f"{instruction}\n\n" + "\n\n".join([f"USER: {inp}\nASSISTANT: {out}" for inp, out in few_shot_examples])

# Load DataFrame

# Find the first empty entry in 'EXA_interpreted'
start_index = 0  # inter_df['EXA_interpreted'].last_valid_index() - 1
print(start_index)
if pd.isna(start_index):  # Check if all rows are filled
    start_index = len(inter_df)  # Set start_index to end if all entries are filled

# Set up the ThreadPoolExecutor
executor = ProcessPoolExecutor(max_workers=10)
futures = []

for idx, row in inter_df.iloc[start_index:].iterrows():
    futures.append(executor.submit(generate_response, idx, row['EXA_formatted'], few_shot_prompt, text_generation_pipeline))

# Collect the results and update the DataFrame
for future in as_completed(futures):
    idx, output_text = future.result()
    inter_df.at[idx, 'EXA_interpreted'] = output_text

# Write the DataFrame to CSV
inter_df.to_csv("../data/interpreted_data2.csv", index=False)


KeyboardInterrupt: 

In [ ]:
inter_df.to_csv("../data/interpreted_data.csv")


In [ ]:
inter_df['EXA_interpreted'].count()

1656

In [ ]:
# import pandas as pd
# from concurrent.futures import ThreadPoolExecutor, as_completed
# def generate_response(idx, text, few_shot_prompt, text_generation_pipeline):
#     if text != "" and text is not None:
#         prompt = f"{few_shot_prompt}\n\nUSER: {text}\nASSISTANT:"
#         generated_text = text_generation_pipeline(prompt, max_new_tokens=200)[0]['generated_text']
#         return idx, generated_text
#     return idx, ""
# # Construct the few-shot prompt
# instruction = "Du bist ein Patient mit den folgenden Symptomen. Ersetze alle lateinischen und deutschen Fachbegriffe und formuliere es umgangssprachlich."
# few_shot_prompt = f"{instruction}\n\n" + "\n\n".join([f"USER: {inp}\nASSISTANT: {out}" for inp, out in few_shot_examples])
# # Load DataFrame
# # Find the first empty entry in 'EXA_interpreted'
# start_index = 0 #inter_df['EXA_interpreted'].last_valid_index() - 1
# print(start_index)
# if pd.isna(start_index):  # Check if all rows are filled
#     start_index = len(inter_df)  # Set start_index to end if all entries are filled
# # Set up the ThreadPoolExecutor
# executor = ThreadPoolExecutor(max_workers=10)
# futures = []
# 
# for idx, row in inter_df.iloc[start_index:].iterrows():
#     futures.append(executor.submit(generate_response, idx, row['EXA_formatted'], few_shot_prompt, text_generation_pipeline))
# 
# # Collect the results and update the DataFrame
# for future in as_completed(futures):
#     idx, output_text = future.result()
#     inter_df.at[idx, 'EXA_interpreted'] = output_text
# 
# # Write the DataFrame to CSV
# inter_df.to_csv("../data/interpreted_data.csv")


In [ ]:
inter_df.head()

In [ ]:
import pandas as pd

#inter_df = pd.read_csv("../data/interpreted_data.csv")
inter_df.head()

In [ ]:
# Construct the few-shot prompt
instruction = "Du bist ein Patient mit den folgenden Symptomen. Ersetze alle lateinischen und deutschen Fachbegriffe und formuliere es umgangssprachlich."
few_shot_prompt = f"{instruction}\n\n" + "\n\n".join([f"USER: {inp}\nASSISTANT: {out}" for inp, out in few_shot_examples])
text = "Sonstige Spontanruptur eines oder mehrerer Bänder des Kniegelenkes: vorderes Kreuzband"
# Combine the text and instruction
prompt = f"{few_shot_prompt}\n\nUSER: {text} \n ASSISTANT:"
# Generate the output
generated_text = text_generation_pipeline(prompt, max_new_tokens=200)[0]['generated_text']
output_text = generated_text
print(text)
print(output_text)

## Labels

In [ ]:
meniskus = [
    "M23.3",
    "S83.2", # Rupture
]
meniskus_low = [
    "M23.0", # Ganglion doesn't seem to urgent
    "M23.1", # something you have from birth
    "M23.2", # problems because of old rupture
    "M23.9"  # Same as M23.8
]
cruciate = [
    "M23.6", # Ruptur
    "S83.50", # might be rupture
    "S83.53", # Rupture
    "S83.54", # Rupture
    "S83.7" # Multiple areas
]
cruciate_low = [
    "M23.8", # Seems to be no rupture, rather weakness, still operations
    "S83.51", # Distorsion
    "S83.52", # Distorsion
]
other_urgent = [
    "S83.3" # Rupture Kniegelenkknorpel
    "S83.4" # Rupture Seitenband
]
other = [
    "M17",
    "M22",  
    "M23.4", # Freikörper, does have a lot of operations
    "M23.5", # instability
    "S83.1", # Only luxation
    "S83.6" # Distorsion somewhere
]